<a href="https://colab.research.google.com/github/alimomennasab/MNIST-Analyzer/blob/main/MNIST_analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#IMPORTS

import torch
import torch.nn as nn                         #all neural network modules & loss functions
import torch.optim as optim                   #optimization algorithms (adam, sigmoid)
import torch.nn.functional as F               #all functions without parameters (activation functions like relu)
from torch.utils.data import DataLoader       #easier dataset management, creates minibatches for training
import torchvision.datasets as datasets       #imports pytorch datasets
import torchvision.transforms as transforms   #transformations/augmentations for dataset

In [ ]:
#CREATING NETWORK

class NN(nn.Module):
    def __init__(self, input_size, num_classes): #input size = 28 x 28 images in dataset = 784 nodes
        super(NN, self).__init__()              #calls initialization method of init class
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, num_classes)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [ ]:
#SETTING DEVICE
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
#HYPERPARAMETERS
input_size = 784
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 1

In [ ]:
#LOADING DATA
train_dataset = datasets.MNIST(root='dataset/', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = datasets.MNIST(root='dataset/', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting dataset/MNIST/raw/train-images-idx3-ubyte.gz to dataset/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting dataset/MNIST/raw/train-labels-idx1-ubyte.gz to dataset/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting dataset/MNIST/raw/t10k-images-idx3-ubyte.gz to dataset/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting dataset/MNIST/raw/t10k-labels-idx1-ubyte.gz to dataset/MNIST/raw



In [ ]:
#INITIALIZE NETWORK
model = NN(input_size=input_size, num_classes=num_classes).to(device)

In [ ]:
#LOSS AND OPTIMIZER
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

In [ ]:
#TRAINING NETWORK
for epoch in range(num_epochs):             #1 epoch = network has seen all the images once
    for batch_idx, (data, targets) in enumerate(train_loader):
        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # Get to correct shape
        data = data.reshape(data.shape[0], -1)

        # Forward
        scores = model(data)
        loss = criterion(scores, targets)

        # Backward
        optimizer.zero_grad()
        loss.backward()

        # Gradient Descent or Adam Step
        optimizer.step()

In [ ]:
#Checking Accuracy
def check_accuracy(loader, model):
    if loader.dataset.train:
        print("Checking accuracy on training data")
    else: print("Checking accuracy on test data")

    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            x = x.reshape(x.shape[0], -1)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        print(f'Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples) * 100:.2f}')

    model.train()

check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Checking accuracy on training data
Got 56028 / 60000 with accuracy 93.38
Checking accuracy on test data
Got 9329 / 10000 with accuracy 93.29
